In [14]:
!pip install Augmentor

Defaulting to user installation because normal site-packages is not writeable


In [4]:
import Augmentor
import numpy as np
from PIL import Image
import random
import shutil
import os

In [5]:
def list_train_test_split(data_dir):
    imgs = []
    for i in sorted(os.listdir(os.path.join(data_dir, "tmap"))):
        img_path = os.path.join(data_dir, "tmap", i)
        if os.path.isfile(img_path):
            imgs.append(img_path)

    train_split = 0.8
    train_samples = random.sample(imgs, int(0.8*len(imgs)))

    with open(os.path.join(data_dir, "test.txt"), "w") as f1:
        with open(os.path.join(data_dir, "train.txt"), "w") as f2:
            for s in imgs:
                if s in train_samples:
                    f2.write(s+"\n")
                else:
                    f1.write(s+"\n")

In [6]:
def move_to_train_test_dirs(trainfile, testfile):
    train, test = [], []
    with open(trainfile, "r") as f:
        for l in f.readlines():
            train.append(l.strip())
    
    with open(testfile, "r") as f:
        for l in f.readlines():
            test.append(l.strip())
    
    shutil.rmtree("ohaze/train")
    shutil.rmtree("ohaze/test")
    
    os.makedirs("ohaze/train/tmap")
    os.makedirs("ohaze/test/tmap")
    os.makedirs("ohaze/train/hazy")
    os.makedirs("ohaze/test/hazy")

    for tr in train:
        shutil.move(tr, f"ohaze/train/tmap/{os.path.split(tr)[1]}")
        shutil.move(tr.replace("tmap", "hazy"), f"ohaze/train/tmap/{os.path.split(tr)[1]}".replace("tmap", "hazy"))
    
    for tr in test:
        shutil.move(tr, f"ohaze/test/tmap/{os.path.split(tr)[1]}")
        shutil.move(tr.replace("tmap", "hazy"), f"ohaze/test/tmap/{os.path.split(tr)[1]}".replace("tmap", "hazy"))
    

In [7]:
def clean_names(data_dir):
    """Makes the file names in the hazy and GT directories 
    the same so that Augmentor detects corresponding files """
    for f in os.listdir(data_dir):
        if "hazy" in f:
            shutil.move(os.path.join(data_dir, f), os.path.join(data_dir, f.replace("_hazy", "")))
        elif "tmap" in f:
            shutil.move(os.path.join(data_dir, f), os.path.join(data_dir, f.replace("_tmap", "")))

In [8]:
list_train_test_split("ohaze")

In [10]:
move_to_train_test_dirs("ohaze/train.txt", "ohaze/test.txt")

In [11]:
clean_names("ohaze/train/hazy")
clean_names("ohaze/train/tmap")
clean_names("ohaze/test/hazy")
clean_names("ohaze/test/tmap")

In [12]:
p = Augmentor.Pipeline("ohaze/train/hazy")
p.ground_truth("ohaze/train/tmap")

Initialised with 36 image(s) found.
Output directory set to ohaze/train/hazy\output.

36 ground truth image(s) found.


In [13]:
p.rotate(probability=0.5, max_left_rotation=10, max_right_rotation=10)
p.flip_left_right(probability=0.8)
p.crop_random(probability=0.5, percentage_area=0.85)
p.random_distortion(probability=0.5, grid_width=4, grid_height=4, magnitude=8)

In [14]:
p.sample(1000)

Processing <PIL.Image.Image image mode=RGB size=3037x2606 at 0x1FE7FED02B0>: 100%|██████████| 1000/1000 [01:30<00:00, 11.07 Samples/s]                 
